In [20]:
#Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from sqlalchemy import create_engine

In [2]:
def create_wiki(cities):
    # An initially empty list that will eventually store one dictionary of information for each city
    list_city_df = []

    # Web Scraping loop to generate a dictionary containing information for each city
    for city in cities:
        # We can take advantage of the consistent structure of Wikipedia URLs to our benefit 
        # All URLs share the same format, differing only in the specific city name
        url = f'https://en.wikipedia.org/wiki/{city}'
        # Here we create the BeautifulSoup object for the city
        r = requests.get(url)
        soup_city = BeautifulSoup(r.content, 'html.parser')

        # Here, we set up our empty dictionary to store information specific to the city
        city_dict = {}

        # Here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
        city_dict['city'] = soup_city.select_one(".firstHeading").get_text()

        # Check if the country information is present
        country_element = soup_city.select_one(".infobox-data")
        city_dict['country'] = country_element.get_text() if country_element else None

        city_dict['latitude'] = soup_city.select_one(".latitude").get_text()
        city_dict['longitude'] = soup_city.select_one(".longitude").get_text()

        # not all Wikipedia pages include elevation information like the city Hamburg
        # the 'if' statement ensures our code can proceed without being halted by this obstacle
        # Checks if elevation information is present on the Wikipedia page and adds it to the dictionary if available
        if soup_city.select_one('.infobox-label:-soup-contains("Elevation")'):
            city_dict['elevation'] = soup_city.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
        
        # append the dictionary for the city to the list_city_df
        list_city_df.append(city_dict)

    # make a DataFrame with the list
    cities_df = pd.DataFrame(list_city_df)

    # Data Cleaning
    cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    cities_df.insert(4, 'elevation_in_meters', cities_df['elevation'].str.extract('(\d+)').astype(float))

    # return the DataFrame
    return cities_df


In [6]:
list_of_cities = ['Berlin', 'Hamburg', 'Munich']
cities_df = create_wiki(list_of_cities)

In [7]:
cities_df

,city,country,latitude,longitude,elevation_in_meters,elevation
0,Berlin,Germany,52.3112,13.2418,34.0,34 m (112 ft)
1,Hamburg,Germany,53.33N,10.00E,NaN,NaN
2,Munich,Germany,48.0815,11.3430,520.0,"520 m (1,710 ft)"


In [26]:
## 5.&nbsp; Sending the information from this notebook to sql 📠

In [ ]:
#Creating the matching city table with SQL
#Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below

In [ ]:
-- DROP DATABASE web_Scrap_wiki_database
-- CREATE DATABASE web_Scrap_wiki_database;
USE web_Scrap_wiki_database;

-- DROP TABLE cities;
CREATE TABLE cities (
    city_id INT AUTO_INCREMENT,
    city VARCHAR(50),
    country VARCHAR(50),
    PRIMARY KEY (city_id)
);

-- DROP TABLE other_facts;
CREATE TABLE other_facts (
    city_id INT,
    latitude FLOAT(6),
    longitude FLOAT(6),
    elevation_in_meters INTEGER,
    elevation VARCHAR(50),
    website VARCHAR(100),
    population INTEGER,
    PRIMARY KEY (city_id), -- Change this line to use city_id as the primary key
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
);cities

In [8]:
host = "127.0.0.1"
user = "root"
password = "ire3812802P$mel$"
port = 3306
database_name = "gans_db"

connection_string_without_db = f'mysql+pymysql://{user}:{password}@{host}:{port}/'
engine_without_db = create_engine(connection_string_without_db)

with engine_without_db.connect() as connection:
    connection.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection_string)

In [ ]:
#Preparing and sending the tables to sql

In [11]:
schema="gans_db"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="admin"
password="ire3812802P$mel$" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'



In [14]:
from decimal import Decimal
# Handle scientific notation in 'longitude' column
def convert_longitude(x):
    try:
        return float(x)
    except ValueError:
        try:
            return float("{:.6f}".format(Decimal(x.replace('E', 'e'))))
        except Exception as e:
            print(f"Error converting {x} to float: {e}")
            return None  # or any default value you prefer

cities_df['longitude'] = cities_df['longitude'].apply(convert_longitude)

# Drop rows where longitude conversion failed
cities_df = cities_df.dropna(subset=['longitude'])

cities_df = cities_df.astype({'latitude': float, 'longitude': float})

cities_df.to_sql('cities_df', con=engine, if_exists='replace', index=False)

2